In [1]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
print(load_dotenv())
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)
logging.langsmith("yong_12")

True
[API KEY]
sk-None-FZcW7iODyxjiAh7qYSpIT3BlbkFJyqnfN***************
LangSmith 추적을 시작합니다.
[프로젝트명]
yong_12


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용

            ]
)
# llm 생성
llm = ChatOpenAI()

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

In [4]:
store = {}

# 세션 id를 기반으로 세션 기록을 가져오는 함수
def get_history(session_ids):
    print(f'[대화 세션ID] : {session_ids}')
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [5]:
chain_with_history.invoke(
    {"question": "나는 한용이야 내가 좋아하는 것은 주지육림이야"},
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID] : abc123


'좋아하는 것을 가지고 즐기는 것은 좋은 일이에요! 주지육림은 맛있는 음식이죠. 한용이에게 주지육림을 주는 것은 좋은 아이디어일 것 같아요. 함께 맛있는 식사를 즐기는 시간이 되기를 바라요.'

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내가 좋아하는게 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID] : abc123


'당신이 좋아하는 것은 주지육림이라고 말씀하셨어요! 주지육림을 좋아한다니 멋진 취향이네요. 맛있는 한 끼 식사를 즐기는 것은 행복한 일이죠. 함께 좋아하는 것들을 찾아 즐기는 시간이 되길 바라요.'

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


In [11]:
loader = PDFPlumberLoader("data/Statistics 11th Edition_240709_201017.pdf")
docs = loader.load()
# 문서분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
# 임베딩
embeddings = OpenAIEmbeddings()
# db생성
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
# 검색기 생성
retriever = vectorstore.as_retriever()
# 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)
# llm생성
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
# 체인 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
store = {}

# 세션 id를 기반으로 세션 기록을 가져오는 함수
def get_history(session_ids):
    print(f'[대화 세션ID] : {session_ids}')
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

chain_with_history.invoke(
    # 질문 입력
    {"question": "ttest의 종류는?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID] : rag123


't-test의 종류는 다음과 같습니다:\n\n1. 단일 표본 t-test (t for one sample)\n2. 두 독립 표본 t-test (t for two independent samples)\n3. 두 관련 표본 t-test (t for two related samples)\n\n이 세 가지 종류의 t-test는 각각의 연구 상황에 따라 적절하게 선택되어 사용됩니다.'

In [13]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "이전 답변을 영어로 번역해주세요."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID] : rag123


'The types of t-tests are as follows:\n\n1. One-sample t-test\n2. Two independent samples t-test\n3. Two related samples t-test\n\nThese three types of t-tests are appropriately selected and used depending on the research situation.'